# 1. Introduction 

Collateral Reblance Pool (CRP) dynamically rebalances Collateral to ensure the ayToken minted (i.e. the loan) remains solvent, especially in an adverse market environment (i.e. the value of the loan does not exceed the value of Collateral). This dynamic rebalancing, together with a careful choice of the key parameters (including loan to Loan-to-Value (LTV) and volatility assumption) allows ALEX to eliminate the liquidation needs. Any residual gap risk (which CRP cannot address entirely) is addressed through maintaining a strong reserve fund. When a Borrower mints ayToken by providing appropriate Collateral, the Collateral is converted into a basket of Collateral and Token, with the weights determined by CRP.

In this notebook, we will help you understand the key attributes of the CRP pool by answering: 
1.  How does CRP achieve the dynamic rebalances with weights determined by CRP? 
2.  How does CRP perform in different market environments? 
3.  How does power arbitrageur play a role in the dynamic rebalances and the bring rebate back to the pool.   
3.  In which parameters space (including LTV, volatility assumption, and power arbitrageur functions) and the CRP would maintain a low default risk and a high pool-value level to a collateral ratio (PVTC)?  

Given there is no close form of CPR performance, we use simulation to show the results based on the predicted future scenarios.  


# 2.  CRP performance by Simulations  

In this session, we want to simulate how CRP performs in different market environments.  Basically, a CRP would serve as an agent (bot) response to the actual market environment by updating the pool weight based on current token price $p$, actual price volatility $\eta$, and estimated price volatility $\sigma$.  We simplify the market environment and let the token price change follow a linear growth trend with variation, formally named the Geometric Brownian Motion (https://en.wikipedia.org/wiki/Geometric_Brownian_motion).  By setting up different growth rates $r$ and volatility $\eta$, we can approximately mimic different market environments.  

Two metrics a liquidity provider (LP) would be very interested to know are 1) The final pool value relative to the collatrals, and 2) the risk of default, i.e., when the LtV >1 at any time point.  We can empirically estimate the default risk and PVTC for any given parameters by conducting Monte Carlo simulations. 

For simplicity, token APY are not considered for now.  Specially we are focsuing on the exponential moving avager (EMA) approach.    
 
We set the following initial conditions: 

Initial weights = 50/50

Loan lifetime = 91 days

LTV0 = 80% 

Realized volatility various from 0.5 to 1.5. 

Assume B-S voliatility equals to Realized volatility. 

EMA factor = 0.95. 


## 3.1 Case 1:  In a BTC upward market, with different realized volatiltiy. 

1a | 1b
- | - 
![alt](./figures/pvtc_by_vol_upward.png) | ![alt](./figures/default_by_vol_upward.png)

## 3.2 Case 2:  In a BTC flat market, with different realized volatiltiy. 
2a | 2b
- | - 
![alt](./figures/pvtc_by_vol_flat.png) | ![alt](./figures/default_by_vol_flat.png)

## 3.3 Case 3:  In a BTC downward market, with different realized volatiltiy. 
3a | 3b
- | - 
![alt](./figures/pvtc_by_vol_downward.png) | ![alt](./figures/default_by_vol_downward.png)

## 3.4 Case 4:  Real BTC upward market (2021-01-01 to 2021-03-31), with different realized volatiltiy.   Obersrved realized vol = 0.40
<img src="./figures/animation_2021-01-01_2021-03-31.gif" width="500" align="center">

4a | 4b
- | - 
![alt](./figures/pvtc_by_vol_2021-01-01_2021-03-31.png) | ![alt](./figures/default_by_vol_2021-01-01_2021-03-31.png)



## 3.5 Case 5:  Real BTC downward market (2021-04-01 to 2021-06-30), with different realized volatiltiy.  Obersrved realized vol = 0.65
<img src="./figures/animation_2021-04-01_2021-06-30.gif" width="500" align="center">

5a | 5b
- | - 
![alt](./figures/pvtc_by_vol_2021-04-01_2021-06-30.png) | ![alt](./figures/default_by_vol_2021-04-01_2021-06-30.png)

## 3.6 Case 6:  Real BTC downward market (2021-04-01 to 2021-06-30), with different realized volatiltiy: use factor 0.7 + 50% rebate 
6a | 6b
- | - 
![alt](./figures/pvtc_by_vol_2021-04-01_2021-06-30_rebate.png) | ![alt](./figures/default_by_vol_2021-04-01_2021-06-30_rebate.png)

## 3.7 Case 7:  Real BTC flat market (2020-06-01 to 2020-08-31), with different realized volatiltiy.  Obersrved realized vol = 0.11

<img src="./figures/animation_2020-06-01_2020-08-31.gif" width="500" align="center">


7a | 7b
- | - 
![alt](./figures/pvtc_by_vol_2020-06-01_2020-08-31.png) | ![alt](./figures/default_by_vol_2020-06-01_2020-08-31.png)

In [4]:
# plot of liquity 
import scipy 
import matplotlib.pyplot as plt
import matplotlib.animation
from matplotlib.widgets import Slider
import matplotlib.ticker as mtick
import seaborn as sns
import numpy as np 
import random
import pandas as pd 
from ipywidgets import *
from scipy.stats import norm
#Import simulation function and class 
%run rbpool_env_v3.ipynb

In [5]:
# an episode example  
t =  np.linspace(91,0,92)/365
Real_vol = 0.75
Growth_rate = 0
LTV0 = 0.8
bs_vol = 0.75
y_price_init = 50000
Collateral = 10000000
pool_init_x = 5000000 
pool_init_y = 100
pool_init_wx = 0.5
fee_rate = 0.0
rebate=0.0
set_random_seed = True  
price_source = [0]
market = 'upward'
example = get_episode_full(t,y_price_init, bs_vol, Growth_rate, Real_vol, Collateral, LTV0, fee_rate, rebate,
                      pool_init_x, pool_init_y, pool_init_wx, price_source,'SM', 0.95)
example.describe()


,t,y_price,weights_ytoken,imp_loss_rebalance,imp_loss_price,x_locked,y_locked,V,coll_with_rebate,rebate,imp_empirical,delta_x,delta_y,ltv_with_rebate,wt_chg,fee,pvtc_rebate,value_loss_rebalance,value_loss_price,value_loss_combined
count,92.000000,92.000000,92.000000,9.200000e+01,92.000000,9.200000e+01,92.000000,92.000000,9.200000e+01,92.0,92.000000,92.000000,92.000000,92.000000,92.000000,92.0,92.000000,92.000000,92.000000,92.000000
mean,0.124658,48320.421236,0.498477,-1.227704e-04,-0.000167,4.799065e+06,99.606688,37055.112357,9.628131e+06,1.0,-0.000309,15146.332998,-0.455060,0.833851,-0.002414,0.0,0.999948,-1140.012983,-1605.884857,-12456.537287
std,0.073156,4607.759060,0.085856,1.557350e-04,0.000209,6.676451e+05,16.199092,47146.937920,5.809350e+05,0.0,0.000444,115283.058617,2.440839,0.049549,0.007300,0.0,0.047230,1393.886121,2008.995231,182927.663709
min,0.000000,41083.903094,0.277901,-5.960678e-04,-0.001214,3.590774e+06,58.134447,5613.232467,8.722175e+06,1.0,-0.002284,-277120.276556,-7.091957,0.726025,-0.016944,0.0,0.880027,-5290.729187,-11588.755520,-439733.807622
25%,0.062329,44980.125479,0.487011,-1.778442e-04,-0.000251,4.449754e+06,97.148944,13788.598965,9.072622e+06,1.0,-0.000421,-52937.049070,-2.077157,0.799035,-0.007729,0.0,0.966858,-1688.955557,-2387.877699,-152178.030556
50%,0.124658,47277.732307,0.510890,-6.630318e-05,-0.000075,4.773988e+06,103.610649,19871.220923,9.628992e+06,1.0,-0.000099,15434.207816,-0.297991,0.830825,-0.001217,0.0,1.003136,-670.977188,-716.820122,5420.063326
75%,0.186986,50696.730637,0.542420,-8.275673e-06,-0.000025,5.044190e+06,107.871230,25398.310706,1.001209e+07,1.0,-0.000039,93048.177284,1.168602,0.881774,0.003641,0.0,1.037185,-79.034722,-225.898779,107106.478140
max,0.249315,60792.838064,0.628828,-1.942636e-08,0.000000,6.393463e+06,124.035268,253952.564315,1.101891e+07,1.0,0.000003,304542.339551,4.561557,0.917202,0.012466,0.0,1.147964,-0.188318,0.000000,453580.186948


In [7]:
# single test 
t =  np.linspace(91,0,92)/365
set_random_seed = False
Growth_rate = -2
fee_rate = 0.00
rebate = 0
s =10
Real_vol = 0.75
bs_vol = 0.75
pool_init_wx = 0.5
fee_rate = 0.0
rebate=0
market = ''
print('LTV0 ={}, bs_vol = {}, Real_vol= {}, and growth_rate ={}'.format(LTV0, bs_vol, Real_vol,Growth_rate))
pd.DataFrame(ltv_simulation(s, 'SM', 0.95)).describe()


LTV0 =0.8, bs_vol = 0.75, Real_vol= 0.75, and growth_rate =0


,0,1,2,3,4,5,6,7,8,9,10,11,12
count,1.000000e+01,10.000000,10.000000,1.000000e+01,10.000000,10.000000,1.000000e+01,1.000000e+01,10.000000,10.000000,10.000000,10.000000,10.000000
mean,8.901103e+06,1.113839,0.400000,8.480007e+06,-124999.422844,-140922.426503,-8.329752e+05,-1.098897e+06,0.006514,-17921.662114,0.320343,0.264440,0.200000
std,1.282575e+06,0.217398,0.516398,3.010994e+06,42319.188666,24202.236871,1.306472e+06,1.282575e+06,0.001113,36341.909509,0.281279,0.290868,0.421637
min,7.637304e+06,0.793438,0.000000,5.340937e+06,-187578.629759,-181880.835430,-2.086400e+06,-2.362696e+06,0.004947,-113406.587983,0.096317,0.037886,0.000000
25%,8.049595e+06,1.027830,0.000000,6.668987e+06,-152700.354169,-156528.444186,-1.689305e+06,-1.950405e+06,0.005573,-12432.596839,0.112804,0.085460,0.000000
50%,8.558279e+06,1.084417,0.000000,7.730423e+06,-120247.087313,-143823.929694,-1.193639e+06,-1.441721e+06,0.006564,0.000000,0.159623,0.120068,0.000000
75%,8.998042e+06,1.230016,1.000000,8.687665e+06,-88123.939218,-125542.015249,-8.070053e+05,-1.001958e+06,0.007388,0.000000,0.459061,0.313295,0.000000
max,1.128711e+07,1.429956,1.000000,1.400388e+07,-76077.703572,-101052.708513,1.603822e+06,1.287108e+06,0.008253,0.000000,0.862491,0.862491,1.000000


In [23]:
set_random_seed = False
np.random.seed(301)
s=5000
t =  np.linspace(91,0,92)/365
Growth_rate = -2
fee_rate = 0.00
rebate = 0
Real_vol = 0.75
bs_vol = 0.75
pool_init_wx = 0.5
fee_rate = 0.0
factor = 0.95
cha_var = 'real_vol'
for factor in [0.7 , 0.75, 0.8 , 0.85, 0.9 , 0.95, 1]:
    print(factor)
    k = 0 
    for i in np.append(0.75,np.arange(0.5,2.51,0.1)):
        k += 1
        Real_vol, bs_vol = i,i
        _sm = pd.DataFrame(ltv_simulation(s, 'SM', factor))
        _sm[cha_var] = '{:.2f}'.format(i)
        if k == 1:
            SM = _sm
        else:
            SM = pd.concat([SM, _sm])
    SM.columns = ['pool_value', 'pvtc', 'default', 'portfolio', 'imp_weight', 'imp_price', 'PNL_price',\
                  'total_pnl','mean_abs_wt_change','loss_at_default', 'wt_default', 'wt_final','default_at_maturity',cha_var]
    SM['loss_at_maturity'] = SM['pool_value']-Collateral*LTV0
    A = sim_summary(SM, ['pool_value', cha_var], plot=0)
    A['variable']= 'pool_value'
    B = sim_summary(SM, ['pvtc', cha_var], plot=0)
    B['variable']= 'pvtc'
    C = sim_summary(SM, ['default', cha_var], plot=0)
    C['variable']= 'default_anytime'
    C2 = sim_summary(SM, ['default_at_maturity', cha_var], plot=0)
    C2['variable']= 'default_at_maturity'
    D = sim_summary(SM, ['imp_weight', cha_var], plot=0) 
    D['variable']= 'IL_from_weight'
    E = sim_summary(SM, ['imp_price', cha_var], plot=0) 
    E['variable']= 'IL_from_price' 
    F = sim_summary(SM, ['total_pnl', cha_var], plot=0) 
    F['variable'] = 'total_pnl'
    G = sim_summary(SM.loc[SM.default==1], ['loss_at_default', cha_var],plot=0)
    G['variable'] = 'loss_at_default'
    G2 = sim_summary(SM.loc[SM.default_at_maturity==1], ['loss_at_maturity', cha_var],plot=0)
    G2['variable'] = 'loss_at_maturity'
    H = sim_summary(SM, ['mean_abs_wt_change', cha_var],plot=0)
    H['variable'] = 'mean_abs_wt_change'
    I = sim_summary(SM, ['portfolio', cha_var], plot=0) 
    I['variable'] = 'BuynHold'
    J = sim_summary(SM.loc[SM.default==1], ['wt_default', cha_var], plot=0, label='0.75') 
    J['variable'] = 'BTC_wt_default'
    K = sim_summary(SM, ['wt_final', cha_var], plot=0, label='0.75' ) 
    K['variable'] = 'BTC_wt_final'

    sim_output = pd.concat([A,B,C,C2,D,E,F,G,G2, H,I,J, K]).reset_index().set_index('variable')
    sim_output.to_csv('SM_factor_{}_down.csv'.format(int(factor*100)))

0.7
0.75
0.8
0.85
0.9
0.95
1


In [ ]:
# post simulation process 
import pandas as pd
import sys
import os

#file_list[0].split('.')[0].split('_')[3]
# ['SM_{}_up.csv'.format(int(i*100)) for i in np.arange(0,1.01,0.1)]
# file_list= ['SM_rep{}.csv'.format(int(i)) for i in np.arange(0,10,1)] 
file_list = ['SM_factor_95_down.csv', 'SM_factor_95_flat.csv', 'SM_factor_95_up.csv']
writer = pd.ExcelWriter('Simulation_check_wt.xlsx') # Arbitrary output name
for csvfilename in file_list:
    _market =csvfilename.split('.')[0].split('_')[3]
    df = pd.read_csv(csvfilename)
    df.to_excel(writer,sheet_name=os.path.splitext(csvfilename)[0])
    workbook = writer.book 
    worksheet = writer.sheets[os.path.splitext(csvfilename)[0]]
    worksheet.insert_image('B15', 'wt_final_by_real_vol_0.75_{}.png'.format(_market))
    worksheet.insert_image('B50', 'wt_default_by_real_vol_0.75_{}.png'.format(_market))

writer.save()   


# import xlsxwriter
# # Create an new Excel file and add a worksheet.
# workbook = xlsxwriter.Workbook('Simulation_check_wt.xlsx')
# worksheet = workbook.add_worksheet()
# worksheet.insert_image('SM_factor_95_down_wt_final', 'wt_final_by_real_vol_0.75_down.png')
# workbook.close()

In [ ]:
import pandas as pd
import sys
import os

file_list= ['SM_95.csv', 'SM_90.csv', 'SM_70.csv','SM_95_down.csv', 'SM_90_down.csv', 'SM_70_down.csv' ]

writer = pd.ExcelWriter('Simulation_by_rebate.xlsx') # Arbitrary output name
for csvfilename in file_list:
    df = pd.read_csv(csvfilename)
    df.to_excel(writer,sheet_name=os.path.splitext(csvfilename)[0])
    result = pd.concat(frames)
writer.save()

In [ ]:

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
#SM.para_value= le.fit_transform(SM.para.values)
SM['para_value']=le.fit_transform(SM.para.values)
labels = SM[['para','para_value']].drop_duplicates().para.values
SM.columns = ['loss', 'V', 'pctc', 'default', 'para', 'para_value']
def plot_boxplt(df, cols, labels=''):
    import matplotlib.pyplot as plt
    import numpy as np
    
    _mean = df.groupby(cols[1]).mean().reset_index()
    _std = df.groupby(cols[1]).std().reset_index()
    fig, ax = plt.subplots(figsize=(8,6))
    ax.errorbar(_mean[cols[1]], _mean[cols[0]], _std[cols[0]], linestyle='None', marker='^')
    ax.set_xlabel(cols[1])
    ax.set_ylabel(cols[0])
    ax.set_title('{} by {}'.format(cols[0], cols[1]))
    plt.show()
plot_boxplt(SM, ['V', 'para_value'], labels)